# Instalamos e importamos librerias necesarias

In [2]:
!pip install praw pandas textblob matplotlib seaborn
import praw
import os
import praw
import pandas as pd
import time
from datetime import datetime

# Guardar resultados en la carpeta output

In [3]:
output_dir = '../output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Carpeta '{output_dir}' creada")
else:
    print(f"Usando carpeta existente: {output_dir}")

print("=== CONFIGURACIÓN COMPLETADA ===")
print(f"Directorio de trabajo: {os.getcwd()}")
print(f"Carpeta de salida: {output_dir}")

Usando carpeta existente: ../output
=== CONFIGURACIÓN COMPLETADA ===
Directorio de trabajo: c:\Users\Usuario\Desktop\CICLOWAA8\data science\web_scrapping\code
Carpeta de salida: ../output


# Configuramos credenciales de Reddit y probamos conexión

In [4]:
REDDIT_CONFIG = {
    'client_id': 'gj_-APZ-9Jb84203m99zIg',          
    'client_secret': 'I2R1lhOwO_Ofy8tpNTjx1H3gpBA-Zg',   
    'username': 'Wonderful-Band-2611',          
    'password': 'pancho11',           
    'user_agent': 'Python:PoliticalSentimentAnalyzer:v1.0 (by /u/Wonderful-Band-2611)'  
}

In [5]:
def initialize_reddit():
    """API Connection (PRAW) - Conectar usando credenciales"""
    try:
        reddit = praw.Reddit(
            client_id=REDDIT_CONFIG['client_id'],
            client_secret=REDDIT_CONFIG['client_secret'],
            user_agent=REDDIT_CONFIG['user_agent'],
            username=REDDIT_CONFIG['username'],
            password=REDDIT_CONFIG['password']
        )
        user = reddit.user.me()
        print(f"Conectado como: {user}")
        return reddit
    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

def test_reddit_connection():
    """Función para probar la conexión antes de empezar"""
    try:
        reddit = praw.Reddit(**REDDIT_CONFIG)
        user = reddit.user.me()
        print(f"Conexión exitosa como: {user}")
        print(f"Karma de comentarios: {user.comment_karma}")
        print(f"Karma de posts: {user.link_karma}")
        
        test_subreddit = reddit.subreddit('politics')
        print(f"Acceso a r/politics: {test_subreddit.subscribers:,} suscriptores")
        return reddit
    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

# EJECUCIÓN PARTE 1
if __name__ == "__main__":
    print("=== PARTE 1: REDDIT API SETUP ===")
    reddit_instance = test_reddit_connection()
    
    if reddit_instance:
        print("API Setup completado correctamente")
    else:
        print("Revisa tu configuración antes de continuar")

=== PARTE 1: REDDIT API SETUP ===
Conexión exitosa como: Wonderful-Band-2611
Karma de comentarios: 0
Karma de posts: 1
Acceso a r/politics: 8,883,094 suscriptores
API Setup completado correctamente


# PARTE 2A: COLLECT POSTS

In [6]:
def collect_posts(reddit):
    """
    Collect Posts from Subreddits (3pts)
    - Target: r/politics, r/PoliticalDiscussion, r/worldnews
    - Task: 20 "hot" posts per subreddit
    - Extract: title, score, num_comments, id, url
    """
    target_subreddits = ['politics', 'PoliticalDiscussion', 'worldnews']
    posts_data = []
    
    print("RECOLECTANDO POSTS...")
    for subreddit_name in target_subreddits:
        try:
            print(f"Procesando r/{subreddit_name}...")
            subreddit = reddit.subreddit(subreddit_name)
            
            for post in subreddit.hot(limit=20):
                posts_data.append({
                    'title': post.title,           # título según tarea
                    'score': post.score,           # upvotes según tarea
                    'num_comments': post.num_comments,  # num_comments según tarea
                    'id': post.id,                 # unique identifier según tarea
                    'url': post.url                # url según tarea
                })
                time.sleep(0.1)
            
            print(f"r/{subreddit_name}: 20 posts recolectados")
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Error en r/{subreddit_name}: {e}")
    
    posts_df = pd.DataFrame(posts_data)
    print(f"Total posts recolectados: {len(posts_df)}")
    return posts_df

# EJECUCIÓN PARTE 2A
if __name__ == "__main__":
    print("\n=== PARTE 2A: COLLECT POSTS (3pts) ===")
    reddit = initialize_reddit()
    if reddit:
        posts_df = collect_posts(reddit)
        
        # Guardar posts temporalmente para Parte 2B
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        posts_df.to_csv(os.path.join(output_dir, f'posts_data_{timestamp}.csv'), index=False)
        posts_df.to_csv(os.path.join(output_dir, 'posts_data.csv'), index=False)
        print("Posts guardados en output/")


=== PARTE 2A: COLLECT POSTS (3pts) ===
Conectado como: Wonderful-Band-2611
RECOLECTANDO POSTS...
Procesando r/politics...
r/politics: 20 posts recolectados
Procesando r/PoliticalDiscussion...
r/PoliticalDiscussion: 20 posts recolectados
Procesando r/worldnews...
r/worldnews: 20 posts recolectados
Total posts recolectados: 60
Posts guardados en output/


# PARTE 2B: COLLECT COMMENTS 

In [7]:
def collect_comments(reddit, posts_df):
    """
    Collect Comments (3pts)
    - Task: For subset of most relevant posts collect 5 comments per post
    - Extract: body, score, post_id
    """
    print("\nRECOLECTANDO COMENTARIOS...")
    
    # Seleccionar posts más relevantes por score
    most_relevant_posts = posts_df.nlargest(15, 'score')
    comments_data = []
    
    for _, post_row in most_relevant_posts.iterrows():
        try:
            print(f"Post: {post_row['title'][:40]}...")
            submission = reddit.submission(id=post_row['id'])
            submission.comments.replace_more(limit=0)
            
            # Recolectar exactamente 5 comentarios como pide la tarea
            comment_count = 0
            for comment in submission.comments:
                if (hasattr(comment, 'body') and 
                    comment.body not in ['[deleted]', '[removed]'] and 
                    comment_count < 5):
                    
                    comments_data.append({
                        'body': comment.body,          # comment text según tarea
                        'score': comment.score,        # upvotes on comment según tarea
                        'post_id': post_row['id']      # link back to original post según tarea
                    })
                    comment_count += 1
            
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Error en post {post_row['id']}: {str(e)[:50]}...")
    
    comments_df = pd.DataFrame(comments_data)
    print(f"Total comentarios recolectados: {len(comments_df)}")
    return comments_df

# EJECUCIÓN PARTE 2B
if __name__ == "__main__":
    print("\n=== PARTE 2B: COLLECT COMMENTS (3pts) ===")
    
    # Cargar posts de Parte 2A
    try:
        posts_df = pd.read_csv(os.path.join(output_dir, 'posts_data.csv'))
        reddit = initialize_reddit()
        if reddit:
            comments_df = collect_comments(reddit, posts_df)
            
            # Guardar comentarios
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            comments_df.to_csv(os.path.join(output_dir, f'comments_data_{timestamp}.csv'), index=False)
            comments_df.to_csv(os.path.join(output_dir, 'comments_data.csv'), index=False)
            print("Comentarios guardados en output/")
    except FileNotFoundError:
        print("Ejecuta primero la Parte 2A")


=== PARTE 2B: COLLECT COMMENTS (3pts) ===
Conectado como: Wonderful-Band-2611

RECOLECTANDO COMENTARIOS...
Post: To defend against Russian tanks, Finland...
Post: Donald Trump Declares D.C. a 'Crime Free...
Post: Donald Trump's 50-Day Deadline to Russia...
Post: Maduro warns of "bloody threat" as Trump...
Post: Trump, Gabbard fired top CIA Russia expe...
Post: Donald Trump to make televised announcem...
Post: Indian PM meets Putin and urges him to s...
Post: Alarm after FBI arrests US army veteran ...
Post: Kremlin denies Trump, Putin ever agreed ...
Post: Trump illegally sent National Guard to L...
Post: France orders hospitals war-ready by Mar...
Post: Nestle abruptly removes CEO Freixe over ...
Post: Ukraine’s Flamingo missile reportedly ma...
Post: Americans Lose Faith That Hard Work Lead...
Post: Bizarre Video Shows Mystery Objects Thro...
Total comentarios recolectados: 75
Comentarios guardados en output/


# PARTE 2C: STORAGE & LINKING 

In [8]:
def store_and_link_data(posts_df, comments_df):
    """
    Storage: Store data linking each comment to its parent post (2pts)
    """
    print("\nGUARDANDO Y VINCULANDO DATOS...")
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Guardar posts con timestamp y archivo fijo
    posts_df.to_csv(os.path.join(output_dir, f'posts_data_{timestamp}.csv'), index=False)
    posts_df.to_csv(os.path.join(output_dir, 'posts_data.csv'), index=False)
    
    # Guardar comentarios (ya contienen post_id para vinculación)
    comments_df.to_csv(os.path.join(output_dir, f'comments_data_{timestamp}.csv'), index=False)
    comments_df.to_csv(os.path.join(output_dir, 'comments_data.csv'), index=False)
    
    # Verificar vinculación según tarea
    posts_with_comments = comments_df['post_id'].nunique()
    total_linkages = len(comments_df)
    
    print("DATOS GUARDADOS Y VINCULADOS:")
    print(f"Posts guardados: {len(posts_df)}")
    print(f"Comentarios guardados: {len(comments_df)}")
    print(f"Posts con comentarios: {posts_with_comments}")
    print(f"Total vinculaciones: {total_linkages}")
    
    # Ejemplo de vinculación
    if not comments_df.empty:
        sample_post_id = comments_df.iloc[0]['post_id']
        sample_post = posts_df[posts_df['id'] == sample_post_id].iloc[0]
        sample_comments = comments_df[comments_df['post_id'] == sample_post_id]
        
        print(f"\nEJEMPLO DE VINCULACIÓN:")
        print(f"Post ID: {sample_post_id}")
        print(f"Título: {sample_post['title'][:50]}...")
        print(f"Comentarios vinculados: {len(sample_comments)}")

# EJECUCIÓN PARTE 2C
if __name__ == "__main__":
    print("\n=== PARTE 2C: STORAGE & LINKING (2pts) ===")
    
    try:
        posts_df = pd.read_csv(os.path.join(output_dir, 'posts_data.csv'))
        comments_df = pd.read_csv(os.path.join(output_dir, 'comments_data.csv'))
        
        store_and_link_data(posts_df, comments_df)
        print("Vinculación completada")
    except FileNotFoundError:
        print("Ejecuta primero las Partes 2A y 2B")


=== PARTE 2C: STORAGE & LINKING (2pts) ===

GUARDANDO Y VINCULANDO DATOS...
DATOS GUARDADOS Y VINCULADOS:
Posts guardados: 60
Comentarios guardados: 75
Posts con comentarios: 15
Total vinculaciones: 75

EJEMPLO DE VINCULACIÓN:
Post ID: 1n5tdc8
Título: To defend against Russian tanks, Finland and Polan...
Comentarios vinculados: 5
Vinculación completada
